In [ ]:
%matplotlib inline
import pandas as pd
import urllib as ur
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np

## Requisitando os carros e filtrando com base do seu tipo(normais, elétricos e híbridos).

In [ ]:
# Requisição a lista de carros e suas informações
response = ur.request.urlopen('https://bookit.modo.coop/api/v2/car_list').read().decode('UTF-8')
json_cars = json.loads(response)

In [ ]:
# Adquirindo os ids de todos os carros
car_ids = list(json_cars['Response']['Cars'].keys())

In [ ]:
electric = []

# Separando por carro o seu tipo, sendo hibrido ou elétrico
for car in car_ids:
    try:
        accessories = list(json_cars['Response']['Cars'][str(car)]['Accessories'])

        if ('hybrid' in accessories):
            electric.append([int(car), 'hybrid'])
        if ('electric car' in accessories):
            electric.append([int(car), 'electric'])
    except:
        continue
        
electric = pd.DataFrame(electric, columns=['car_id', 'type'])

## Coletando a duração de cada viagem para carros normais, elétricos e híbridos.

In [ ]:
# Lendo csv de intervalos de ociosidade e ocupação
dfOccupied = pd.read_csv('occupied_v2.csv')
dfParked = pd.read_csv('parked_v2.csv')

In [ ]:
def Hour_Diff(h1,h2):
    """
    Faz a diferença entre duas horas dadas e retorna em minutos
    
    Parameters
    ----------
    h1, h2 : unix timestamp
        Hora inicio e fim para ser feito o cálculo da diferença
        
    Returns
    ---------
    diff : float
        Diferença entre as duas horas dadas em minutos
        
    """
    
    h1Aux = datetime.datetime.fromtimestamp(h1)
    h2Aux = datetime.datetime.fromtimestamp(h2)
    diff = abs((h1Aux - h2Aux)).total_seconds()/60
    
    return diff

In [ ]:
# Fazendo a duração de cada intervalo de ocupação
diff = []
for i in range(len(dfOccupied)):
    diff.append(Hour_Diff(dfOccupied['start'].iloc[i], dfOccupied['end'].iloc[i]))

diff = pd.DataFrame(diff, columns=['duration'])
Occupied_Diff = pd.concat([dfOccupied, diff], axis=1)

In [ ]:
# Fazendo a duração de cada intervalo de tempo de ociosidade
diff = []
for i in range(len(dfParked)):
    diff.append(Hour_Diff(dfParked['start'].iloc[i], dfParked['end'].iloc[i]))

diff = pd.DataFrame(diff, columns=['duration'])
Parked_Diff = pd.concat([dfParked, diff], axis=1)

In [ ]:
# Filtra os intervalos de ocupação dos carros híbridos e elétricos

electric_occupied = []
hybrid_occupied = []

for i in range(len(electric)):
    occupied = Occupied_Diff[Occupied_Diff['car_id'] == electric['car_id'].iloc[i]]

    if (electric['type'].iloc[i] == 'hybrid'):
        for j in range(len(occupied)):
            hybrid_occupied.append([occupied['car_id'].iloc[j] ,occupied['start'].iloc[j], 
                                   occupied['end'].iloc[j],occupied['duration'].iloc[j]])
        continue
    if (electric['type'].iloc[i] == 'electric'):
        for j in range(len(occupied)):
            electric_occupied.append([occupied['car_id'].iloc[j] ,occupied['start'].iloc[j], 
                                     occupied['end'].iloc[j], occupied['duration'].iloc[j]])
        continue

hybrid_occupied = pd.DataFrame(hybrid_occupied, columns=['car_id', 'start', 'end', 'duration'])
electric_occupied = pd.DataFrame(electric_occupied, columns=['car_id', 'start', 'end', 'duration'])

# Filtra o tempo de ociosidade dos carros híbridos e elétricos

electric_parked = []
hybrid_parked = []

for i in range(len(electric)):
    parked = Parked_Diff[Parked_Diff['car_id'] == electric['car_id'].iloc[i]]

    if (electric['type'].iloc[i] == 'hybrid'):
        for j in range(len(parked)):
            hybrid_parked.append([parked['car_id'].iloc[j] ,parked['start'].iloc[j], 
                                  parked['end'].iloc[j], parked['duration'].iloc[j]])
        continue
    if (electric['type'].iloc[i] == 'electric'):
        for j in range(len(parked)):
            electric_parked.append([parked['car_id'].iloc[j] ,parked['start'].iloc[j], 
                                    parked['end'].iloc[j], parked['duration'].iloc[j]])
        continue

hybrid_occupied = pd.DataFrame(hybrid_occupied, columns=['car_id', 'start', 'end', 'duration'])
electric_occupied = pd.DataFrame(electric_occupied, columns=['car_id', 'start', 'end', 'duration'])
hybrid_parked = pd.DataFrame(hybrid_parked, columns=['car_id', 'start', 'end', 'duration'])
electric_parked = pd.DataFrame(electric_parked, columns=['car_id', 'start', 'end', 'duration'])

In [ ]:
# Filtrando tempo de ocupação e tempo de ociosidade de carros normais
normal_cars_occupied = []

for i in range(len(Occupied_Diff)):
    if (Occupied_Diff['car_id'].iloc[i] not in list(electric['car_id'])):
        normal_cars_occupied.append([Occupied_Diff['car_id'].iloc[i], Occupied_Diff['start'].iloc[i], 
                                    Occupied_Diff['end'].iloc[i], Occupied_Diff['duration'].iloc[i]])

normal_cars_occupied = pd.DataFrame(normal_cars_occupied, columns=['car_id', 'start', 'end', 'duration'])


normal_cars_parked = []

for i in range(len(Parked_Diff)):
    if (Parked_Diff['car_id'].iloc[i] not in list(electric['car_id'])):
        normal_cars_parked.append([Parked_Diff['car_id'].iloc[i], Parked_Diff['start'].iloc[i], 
                                    Parked_Diff['end'].iloc[i], Parked_Diff['duration'].iloc[i]])

normal_cars_parked = pd.DataFrame(normal_cars_parked, columns=['car_id', 'start', 'end', 'duration'])


## Plotagem dos CDFs.

In [ ]:
def cdf(df):
    """
    Calcula valores de todas as coordenadas (x,y) para uma CDF.
    
    Parameters
    -----------
    df : float list, pandas.DataFrame
        Lista de valores de durações para ser feita a CDF.
    
    Returns
    ---------
    x, y: float list
        Lista de todas as coordenadas da CDF.
        
    """
    
    df = df.sort_values(by='duration')
    values = df['duration']
    
    x = []
    y = []

    total = float(len(values))
    cnt = 0
    last = values.iloc[0]
    for data in values:
        if data != last:
            x.append(last)
            y.append(cnt/total)
            cnt += 1
            last = data
        else:
            cnt += 1
    x.append(last)
    y.append(cnt/total)
    
    return x, y

In [ ]:
# CDF Ocupado de carros elétricos 
x, y = cdf(electric_occupied.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# CDF Ociosidade de carros elétricos
x, y = cdf(electric_parked.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# CDF Ocupação de carros Híbridos
x, y = cdf(hybrid_occupied.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# CDF Ociosidade de carros Híbridos
x, y = cdf(hybrid_parked.sort_values(by=['duration']))
plt.plot(x,y)

In [ ]:
# CDF Ocupação de carros normais
x, y = cdf(normal_cars_occupied.sort_values(by=['duration']))
plt.plot(x,y)

In [ ]:
# CDF Viagens de carros normais
x, y = cdf(normal_cars_parked.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# Lendo dados após fitting

ocioso_pareto = pd.read_csv('ocioso_pareto.csv', usecols=[1,2])
ocioso_weibull = pd.read_csv('ocioso_weibull.csv', usecols=[1,2])
ocupado_pareto = pd.read_csv('ocupado_pareto.csv', usecols=[1,2])
ocupado_weibull = pd.read_csv('ocupado_weibull.csv', usecols=[1,2])

In [ ]:
# Selecionando somente os valores menores ou iguais a 5400 minutos

normal_cars_occupied = normal_cars_occupied[normal_cars_occupied['duration'] <= 5400]
normal_cars_parked = normal_cars_parked[normal_cars_parked['duration'] <= 5400]
electric_occupied = electric_occupied[electric_occupied['duration'] <= 5400]
electric_parked = electric_parked[electric_parked['duration'] <= 5400]
hybrid_occupied = hybrid_occupied[hybrid_occupied['duration'] <= 5400]
hybrid_parked = hybrid_parked[hybrid_parked['duration'] <= 5400]

In [ ]:


fig, (ax1, ax2) = plt.subplots(1,2)

fig.set_size_inches(14,4.5)

# Plot do tempo ocupado

x, y = cdf(normal_cars_occupied.sort_values(by=['duration']))
ax1.plot(x,y, label='Comuns', marker='o', markevery=[50, 300, 750, 1700, 3000, 4000, 4300, 4500, 4620])


x, y = cdf(electric_occupied.sort_values(by='duration'))
ax1.plot(x,y, label='Elétricos', marker='^', markevery=[5, 60, 95, 130, 145, 155, 158, 160])


x, y = cdf(hybrid_occupied.sort_values(by='duration'))
ax1.plot(x,y, label='Híbridos', marker='s', markevery=[150, 450, 900, 1400, 1800, 2000, 2070, 2080])

ax1.plot(ocupado_pareto['x'], ocupado_pareto['y'], label='Fitted Pareto', linestyle='--')

ax1.plot(ocupado_weibull['x'], ocupado_weibull['y'], label='Fitted Weibull', c='black', linestyle=':')

ax1.grid(b=True, linestyle='--')

# Modificando os labels dos minutos
ax1.xaxis.set_ticks(np.arange(0, 6000, 720))

fig.canvas.draw()

labels = [item.get_text() for item in ax1.get_xticklabels()]
labels = range(0,100,12)

ax1.set_xticklabels(labels)

ax1.legend(bbox_to_anchor=(0.65, 0.35), loc=2, borderaxespad=0.2)
ax1.set_ylabel('ECDF')
ax1.set_xlabel('Tempo Ocupado [horas]')


# Plot do tempo ocioso

x, y = cdf(normal_cars_parked.sort_values(by='duration'))
ax2.plot(x,y, label='Comuns', marker='o', markevery=[100, 900, 1600, 3400, 5200, 7200, 7700, 8230, 8450])

x, y = cdf(electric_parked.sort_values(by='duration'))
ax2.plot(x,y, label='Elétricos', marker='^', markevery=[5, 40, 75, 90, 110, 130, 145, 162, 170])

x, y = cdf(hybrid_parked.sort_values(by='duration'))
ax2.plot(x,y, label='Híbridos', marker='s', markevery=[25, 350, 1200, 1900, 2700, 3120, 3180])

ax2.plot(ocioso_pareto['x'], ocioso_pareto['y'], label='Fitted Pareto', linestyle='--')

ax2.plot(ocioso_weibull['x'], ocioso_weibull['y'], label='Fitted Weibull', c='black', linestyle=':')

ax2.grid(b=True, linestyle='--')

# Modificando os labels dos minutos para horas
ax2.xaxis.set_ticks(np.arange(0, 6000, 720))

fig.canvas.draw()

labels = [item.get_text() for item in ax2.get_xticklabels()]
labels = range(0,100,12)

ax2.set_xticklabels(labels)

ax2.legend(bbox_to_anchor=(0.65, 0.35), loc=2, borderaxespad=0.2)
ax2.set_ylabel('ECDF')
ax2.set_xlabel('Tempo Ocioso [horas]')

plt.savefig('CDFs.pdf', bbox_inches='tight')

## Tarde e noite para dias de semana

In [ ]:
tarde = []
noite = []

for i in range(len(Occupied_Diff)):
    start = Occupied_Diff['start'].iloc[i]
    hora = datetime.datetime.fromtimestamp(start).hour
    dia = int(datetime.datetime.fromtimestamp(start).strftime('%w'))
    
    if ((hora >= 11 and hora <= 13) and (dia > 0  and dia < 6)):
        tarde.append([Occupied_Diff['car_id'].iloc[i], Occupied_Diff['start'].iloc[i], 
                      Occupied_Diff['duration'].iloc[i]])
    
    elif ((hora >= 18 and hora <= 19) and (dia > 0  and dia < 6)):
        noite.append([Occupied_Diff['car_id'].iloc[i], Occupied_Diff['start'].iloc[i], 
                      Occupied_Diff['duration'].iloc[i]])

tarde = pd.DataFrame(tarde, columns=['car_id', 'start', 'duration'])
noite = pd.DataFrame(noite, columns=['car_id', 'start', 'duration'])

In [ ]:
# Durações menores que 66 horas
tarde = tarde[tarde['duration'] < 4000]
noite = noite[noite['duration'] < 4000]

In [ ]:
x, y = cdf(tarde.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
x, y = cdf(noite.sort_values(by='duration'))
plt.plot(x,y)

In [ ]:
# Durações menores ou iguais que 28 horas
tarde = tarde[tarde['duration'] <= 28*60]
noite = noite[noite['duration'] <= 28*60]
todos = Occupied_Diff[Occupied_Diff['duration'] <= 28*60]

In [ ]:
import matplotlib

matplotlib.rc('font', size=12)

fig, ax = plt.subplots()

x, y = cdf(tarde.sort_values(by='duration'))
ax.plot(x,y, label='11h a 13h', marker='o')#, markevery=[30,140,280,400,580,800,960,1010,1100])

x, y = cdf(noite.sort_values(by='duration'))
ax.plot(x,y, label='18h a 19h', marker='s')#, markevery=[30,60,120,190,280,340,392,400,406])

x, y = cdf(todos.sort_values(by='duration'))
ax.plot(x,y, label='Dia todo', marker='^')#, markevery=[20,120,350,600,960,1400,2200,2800,3600,4300,4550])

# Modificando os labels dos minutos
ax.xaxis.set_ticks(np.arange(0, 1800, 180))

fig.canvas.draw()

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = range(0,30,3)

ax.set_xticklabels(labels)

ax.legend(bbox_to_anchor=(0.65, 0.3), loc=2, borderaxespad=0.2)
ax.set_ylabel('ECDF')
ax.set_xlabel('Tempo Ocupado [horas]')

plt.savefig('tarde_noiteCDF.pdf')